# With ChatModel

In [58]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser

In [59]:
def save_result(output, prompt_type, model, init, best_practice, output_parser):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice}
    result['output_parser'] = output_parser 
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [60]:
def StringToJSON(string):
    if string[0] != '{':
        string = "{"+string+"}"

    json_object = json.loads(string)
    return json_object

In [61]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

In [5]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

## Custom Parsers

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field, validator
# from langchain.output_parsers import PydanticOutputParser

# class ListItem(BaseModel):
#     """Item of a list"""

#     id: int = Field(description="The item identifier")
#     content: str = Field(description="The item content")

#     @validator("id")
#     def validate_id(cls, field):
#         if not field:
#             raise ValueError("Item ID cannot be empty!")
#         return field
    
#     @validator("content")
#     def validate_content(cls, field):
#         if not field:
#             raise ValueError("Item content cannot be empty!")
#         return field
    
# parser = PydanticOutputParser(pydantic_object=ListItem)
# ...
# bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format(format_instructions=parser.get_format_instructions())

## Summary Length

In [19]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'summaryLengthPrompt_V1.0.3.json'
bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min= 6, max= 10)

In [20]:
chain = init_prompt | model | StrOutputParser()
print(init_prompt.format(role="Software Engineer", best_practice=bestPractice_prompt, ticket=json_sample))

 ### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Revise a ticket for compliance with these best practices and offer recommendations if needed.
The best practice to check is: 

Best Practice: Summary Length
Your Task is to check if the ticket summary is between 6 and 10 words.
If the word count is in this range, return the original summary.
Otherwise, recommend a new summary for the ticket.
Please return only the summary and nothing else.


### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp and content.
Components: Project components to which the issue belongs.
CreatedDate: The time and d

In [21]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

"'Sourcetree crashes upon opening repeatedly'"

In [25]:
save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')


In [31]:
# # maybe usefull for later
# def run(model_name, init_prompt_name, sumLen_prompt_name):
#     model = ChatOpenAI(
#         model=model_name,
#         api_key=open('api.txt', 'r').read(),
#         temperature=0
#     )
#     with open('./data/json/last_row.json') as f:
#         json_sample = json.load(f)

#     init_prompt = load_prompt('prompts/init/'+init_prompt_name)
#     sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)
    
#     chain = init_prompt | model | StrOutputParser()

#     output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})

#     save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)  

# run('gpt-4', 'initPrompt_V1.0.0.json', 'summaryLengthPrompt_V1.0.1.json')  

## Description Length

In [26]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionLengthPrompt_V1.0.1.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format(min= 29, max= 111)

chain = init_prompt | model | StrOutputParser()

In [27]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

'The current description: "Source tree crashes everytime i open , I have uninstalled and installed many times"\n\nThis description is too short with only 12 words. A more detailed description is recommended for better understanding of the issue.\n\nRecommended description: "I am experiencing an issue where Sourcetree crashes every time I attempt to open it. This has been a consistent problem, and I have tried to resolve it by uninstalling and reinstalling the software multiple times, but the issue persists. The version I am using is 3.4.4. This problem is hindering my work as I am unable to access my repositories. Any assistance in resolving this issue would be greatly appreciated."'

In [28]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Field Update

In [6]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.11.0.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

In [9]:
chain = init_prompt | model | StrOutputParser()

In [10]:
output = chain.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})

In [22]:
output_json = StringToJSON(output)

In [23]:
save_result(output_json, 'update', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Toxic Speech Detection

In [53]:
with open('./data/json/last_row_toxic.json') as f:
    toxic_sample = json.load(f)

In [54]:
init_prompt_name = 'initPrompt_V1.1.1.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.2.1.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

chain = init_prompt | model | StrOutputParser()

In [55]:
output = chain.invoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":toxic_sample})
output

'"results": [\n    {\n        "comment": "Issue was fixed in latest Sourcetree versions you idiot. Use your brain and use Sourcetree 3.4.5 > ",\n        "author": "Oleksandr Naumenko",\n        "datetime": "2021-11-18 15:12:59.861000+00:00",\n        "recommendation": "The comment contains toxic speech. It is recommended to revise the comment to maintain a respectful and professional tone. For example, \'The issue was fixed in the latest Sourcetree versions. Please consider using Sourcetree 3.4.5 or later.\'"\n    }\n]'

In [56]:
output_json = StringToJSON(output)

In [57]:
save_result(output_json, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name, 'StrOutputParser')

## Bug Report Structure

In [63]:
bugReportStrucutre_prompt_name = 'bugReportStructurePrompt_V1.0.0.json'
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+bugReportStrucutre_prompt_name)

In [64]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [ ]:
output = chain.invoke({"role":"Software Engineer", "bug_Report":json_sample})

In [ ]:
save_result(output_json, 'update', model_name, '-', bugReportStrucutre_prompt_name, 'StrOutputParser')

## Internationalization